In [ ]:
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import socket
import json

In [ ]:
consumer_key='DgAMexqyHxuUfQOOcAMk2i5cJ'
consumer_secret='471aOYht2lIPvyXXVDYF3gdPacGuD1TYf5t9KboztTy5NjEdTM'
access_token ='1521023485287362560-l8LK75rEK9s70UCoTI4DXN7asAwpGA'
access_secret='7L6hHM69d8PmllcuVG1kFMQCLnny38WC3RpjWf4nR1nh5'

In [ ]:
class TweetsListener(tweepy.Stream):
  # tweet object listens for the tweets
  def __init__(self, csocket):
    self.client_socket = csocket
  def on_data(self, data):
    try:  
      msg = json.loads( data )
      print("new message")
      # if tweet is longer than 140 characters
      if "extended_tweet" in msg:
        # add at the end of each tweet "t_end" 
        self.client_socket\
            .send(str(msg['extended_tweet']['full_text']+"t_end")\
            .encode('utf-8'))         
        print(msg['extended_tweet']['full_text'])
      else:
        # add at the end of each tweet "t_end" 
        self.client_socket\
            .send(str(msg['text']+"t_end")\
            .encode('utf-8'))
        print(msg['text'])
      return True
    except BaseException as e:
        print("Error on_data: %s" % str(e))
    return True
  def on_error(self, status):
    print(status)
    return True

In [ ]:
def sendData(c_socket, keyword):
  print('start sending data from Twitter to socket')
  # authentication based on the credentials
  auth = OAuthHandler(consumer_key, consumer_secret)
  api= auth.set_access_token(access_token, access_secret)
  # start sending data from the Streaming API 
  twitter_stream = Stream(api, TweetsListener(c_socket))
  twitter_stream.filter(track = keyword, languages=["id"])

In [ ]:
if __name__ == "__main__":
    # server (local machine) creates listening socket
    s = socket.socket()
    host = "localhost"    
    port = 5555
    s.bind((host, port))
    print('socket is ready')
    # server (local machine) listens for connections
    s.listen(4)
    print('socket is listening')
    # return the socket and the address on the other side of the connection (client side)
    c_socket, addr = s.accept()
    print("Received request from: " + str(addr))
    # select here the keyword for the tweet data
    sendData(c_socket, keyword = ['pemilu2024'])

socket is ready
socket is listening
